In [1]:
from selenium import webdriver
import time
from selenium.common.exceptions import NoSuchElementException


In [2]:
driver = webdriver.Chrome('/Users/mac/projects/ChromeWebDriver/chromedriver')

In [35]:
import time

from selenium import webdriver


def find_apartment_urls():
    driver = webdriver.Chrome('/Users/mac/projects/ChromeWebDriver/chromedriver')
    driver.get('https://www.dabangapp.com/search#/map')

    driver.find_element_by_name('keyword').send_keys('성수동1가')
    time.sleep(2)

    button = driver.find_element_by_xpath('/html/body/div[1]/div/div[1]/div/div[1]/ul/li[1]/ul/li[1]')
    driver.execute_script("arguments[0].click();", button)
    time.sleep(2)

    button = driver.find_element_by_xpath('/html/body/div[1]/div/div[1]/div/div[2]/div[1]/div')
    driver.execute_script("arguments[0].click();", button)
    time.sleep(2)

    button = driver.find_element_by_xpath(
        '/html/body/div[1]/div/div[1]/div/div[2]/div[1]/div[2]/ul[2]/li/label/span[2]')
    driver.execute_script("arguments[0].click();", button)
    time.sleep(2)

    apt_list = []
    for i in range(6):
        apart_urls = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div[2]/ul/li/div/a')
        for url in apart_urls:
            apt_list.append(url.get_attribute('href'))
        button = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div[2]/div/div/div/button[2]')
        driver.execute_script("arguments[0].click();", button)
        time.sleep(2)

    # 성수 2동 시작.
    driver.find_element_by_name('keyword').send_keys('성수동2가')
    time.sleep(2)

    button = driver.find_element_by_xpath('/html/body/div[1]/div/div[1]/div/div[1]/ul/li[1]/ul/li[1]')
    driver.execute_script("arguments[0].click();", button)
    time.sleep(2)

    button = driver.find_element_by_xpath('/html/body/div[1]/div/div[1]/div/div[2]/div[1]/div')
    driver.execute_script("arguments[0].click();", button)
    time.sleep(2)

    apart_urls = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div[2]/ul/li/div/a')

    for url in apart_urls:
        apt_list.append(url.get_attribute('href'))
    button = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div[2]/div/div/div/button[2]')
    driver.execute_script("arguments[0].click();", button)
    time.sleep(2)

    apart_urls = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div[2]/ul/li/div/a')
    for url in apart_urls:
        apt_list.append(url.get_attribute('href'))

    return apt_list


def find_urls():
    driver = webdriver.Chrome('/Users/mac/projects/ChromeWebDriver/chromedriver')
    driver.implicitly_wait(3)
    driver.get('https://www.dabangapp.com/search#/map')
    driver.find_element_by_name('keyword').send_keys('성수동1가')
    driver.find_element_by_xpath('/html/body/div[1]/div/div[1]/div/div[1]/ul/li[1]/ul/li[1]').click()
    driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div[1]/a[1]').click()

    time.sleep(2)
    url_all_list = []
    for i in range(0, 3):
        time.sleep(2)
        seongsoo_1dong_detail_list = driver.find_elements_by_xpath(
            "/html/body/div[1]/div/div[2]/div[1]/div/div[2]/ul/li/div/a")
        for data in seongsoo_1dong_detail_list:
            url_all_list.append(data.get_attribute('href'))
        button = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div[2]/div/div/div/button[2]')
        driver.execute_script("arguments[0].click();", button)
    print(len(url_all_list))

    driver.find_element_by_name('keyword').send_keys('성수동2가')
    time.sleep(2)
    driver.find_element_by_xpath('/html/body/div[1]/div/div[1]/div/div[1]/ul/li[1]/ul/li[1]').click()
    driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div[1]/a[1]').click()
    time.sleep(2)
    seongsoo_2dong_detail_list = driver.find_elements_by_xpath(
        "/html/body/div[1]/div/div[2]/div[1]/div/div[2]/ul[2]/li/div/a")
    for data in seongsoo_2dong_detail_list:
        url_all_list.append(data.get_attribute('href'))
    for i in range(0, 2):
        time.sleep(2)
        seongsoo_2dong_detail_list = driver.find_elements_by_xpath(
            "/html/body/div[1]/div/div[2]/div[1]/div/div[2]/ul/li/div/a")
        for data in seongsoo_2dong_detail_list[2:]:
            url_all_list.append(data.get_attribute('href'))
        button = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div[1]/div/div[2]/div/div/div/button[2]')
        driver.execute_script("arguments[0].click();", button)
    print(len(url_all_list))
    driver.close()
    return url_all_list


In [36]:
url_all_list = find_apartment_urls()
url_all_list += find_urls()

56
126


In [37]:
url_all_list

['https://www.dabangapp.com/room/5e1d6b0773d078168238f1e0',
 'https://www.dabangapp.com/room/5e6857600f04a6772796a407',
 'https://www.dabangapp.com/room/5e818950279b951d62cb4760',
 'https://www.dabangapp.com/room/5e868a4369367f4ba04cbd9a',
 'https://www.dabangapp.com/room/5e7c2481a50ed345f63c82d2',
 'https://www.dabangapp.com/room/5e7c715f773e22686b15ee5c',
 'https://www.dabangapp.com/room/5e8aaa98521b226396ccdbbf',
 'https://www.dabangapp.com/room/5e8d72f43ace3f54748ac195',
 'https://www.dabangapp.com/room/5e6f0f71bcde1c6b698fdc41',
 'https://www.dabangapp.com/room/5e8ae5d9ff822c19ad4d0911',
 'https://www.dabangapp.com/room/5e6c6cced93974472c59c6b8',
 'https://www.dabangapp.com/room/5e81a1b829cfcc06f34d07af',
 'https://www.dabangapp.com/room/5e71aec6af6ef63ffd56a238',
 'https://www.dabangapp.com/room/5e8812ec29ec2d636a1d92ca',
 'https://www.dabangapp.com/room/5e8aa2bf50b6fd52a0957c27',
 'https://www.dabangapp.com/room/5e71a7797741213bb1749b0d',
 'https://www.dabangapp.com/room/5e8d687

In [3]:
driver.get('https://www.dabangapp.com/room/5e804ed6a87412151a8efd35')

# 아파트
# driver.get('https://www.dabangapp.com/room/5e8c1f6ef9ae1d50cbdf754b')

In [4]:
time.sleep(2)
post_type = driver.find_element_by_xpath('/html/body/div[1]/div/div[1]/div/ul/li[1]/p/span')
post_type = post_type.get_attribute('innerText')
post_type

'아파트'

In [5]:
# Broker
try:
    button = driver.find_element_by_xpath("/html/body/div[1]/div/div[1]/div/ul/li/button")
    driver.execute_script("arguments[0].click();", button)
    
    name = driver.find_element_by_xpath('/html/body/div[4]/div/div/div/div[1]/div[1]/h1')
    name = name.get_attribute('innerText')
    print(name)
    address = driver.find_element_by_xpath('/html/body/div[4]/div/div/div/div[1]/div[1]/p')
    address = address.get_attribute('innerText')
    print(address)
    manager = driver.find_element_by_xpath('/html/body/div[4]/div/div/div/div[2]/div[2]/p[1]')
    manager = manager.get_attribute('innerText')
    print(manager)
    tel = driver.find_element_by_xpath('/html/body/div[4]/div/div/div/div[2]/div[2]/p[2]')
    tel = tel.get_attribute('innerText')
    print(tel)
    print(name, address, manager, tel)
except NoSuchElementException:
    manager = driver.find_element_by_xpath('/html/body/div[4]/div/div/div/div/div[2]/p[1]')
    manager = manager.get_attribute('innerText')
    tel = driver.find_element_by_xpath('/html/body/div[4]/div/div/div/div/div[2]/p[2]')
    tel = tel.get_attribute('innerText')
    print(manager)
    print(tel)
button = driver.find_element_by_xpath("/html/body/div[4]/div/div/header/button")
driver.execute_script("arguments[0].click();", button)

park (직거래)
010-2049-3514


In [6]:
# 상세 설명 보기
try:
    button = driver.find_element_by_xpath("/html/body/div[1]/div/div[4]/div/div/button")
    driver.execute_script("arguments[0].click();", button)
except NoSuchElementException:
    pass


In [7]:
unrefined_description = driver.find_elements_by_xpath("/html/body/div[1]/div/div[4]/div/div")
description = unrefined_description[0].get_attribute("innerText")
description.replace("\n", "")

'올수리 정남향 압구정강남교통 편리하나은행융자 잔액2억1천정도만남기고 전세계약 원합니다ㆍ 감액등기해서 전세보증보혐로도 대납해줘서 임차인걱정없고요 린나이보일러도70주고작년설치 및ㅈ거실확장하고 올수리되어잏으며 작은방하나에 ㄷ자형책상및서재만들어져있습니다  입주는4-15일부터가능하며 빠르신분우대합니다'

In [8]:
unrefined_salesform = driver.find_elements_by_xpath('/html/body/div[1]/div/div[1]/div/ul/li[1]/div')
salesForm = unrefined_salesform[0].get_attribute("innerText")
salesForm = salesForm.replace('/', ' ')
salesForm = salesForm.replace('\n', '')
salesForm = salesForm.split()
salesType = salesForm[0]  # sales type
print('salesType: 매물 종류는>>', salesType)
salesDepositChar = salesForm[1]
if salesDepositChar.find('원'):
    salesDepositChar = salesDepositChar.replace('원', '')
    print('salesDepositChar: ', salesDepositChar)
salesdepositInt = salesDepositChar.replace('억', '00000000')
if salesdepositInt.find('만'):
    salesdepositInt = salesdepositInt.replace('만', '')
salesdepositInt = int(salesdepositInt)
print('salesdepositInt: ', salesdepositInt)


try:
    salesmonthlyChar = salesForm[2]
    print('salesmonthlyChar: ', salesmonthlyChar)
    salesmonthlyInt = salesmonthlyChar.replace('만원', '0000')
    salesmonthlyInt = int(salesmonthlyInt)
    print('salesmonthlyInt: ', salesmonthlyInt)
    if salesType == '전세':
        # 전세는 금액이 억, 만원이 붙어 있는 경우가 있어서 이렇게 처리.
        salesdepositInt = salesdepositInt + salesmonthlyInt
        salesDepositChar = salesDepositChar + salesmonthlyChar
except IndexError:
    salesmonthlyInt = 0
    salesmonthlyChar = ''

salesType: 매물 종류는>> 전세
salesDepositChar:  4억
salesdepositInt:  400000000
salesmonthlyChar:  5000만원
salesmonthlyInt:  50000000


In [9]:
if post_type =="아파트":
    if salesType == "매매":
        address = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[3]/div/p')
    else:
        address = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[5]/div/p')
else:
    address = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[4]/div/p')
if not address:
    address = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div/div/p')
print(address)
address = address[0].get_attribute('innerText')
address

[<selenium.webdriver.remote.webelement.WebElement (session="75b37b2ef5ff8fe3fb247349b135cae0", element="0.28095169457930247-8")>]


'서울시 성동구 행당동 344'

In [10]:
unrefined_floor = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[1]/div')
total_floor = unrefined_floor[0].get_attribute('innerText')
total_floor = total_floor.split('/')
floor = total_floor[0]
floor = floor
print('floor>>', floor)
totalFloor = total_floor[1]
totalFloor = totalFloor.replace(' ', '')
totalFloor = totalFloor
print('totalFloor >> ', totalFloor)

floor>> 2층 
totalFloor >>  15층


In [11]:
unrefined_area = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[2]/div/span')
area = unrefined_area[0].get_attribute('innerText')
print('area >>', area)
areaChar = area
print('areaChar >> ', areaChar)

area >> 84.99 / 102.07㎡
areaChar >>  84.99 / 102.07㎡


In [12]:
# 평수로 변환해주는 버튼 클릭
time.sleep(1)
driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[2]/div/button').click()


In [13]:
unrefined_area = driver.find_elements_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[2]/div/span')
supplyAreaChar = unrefined_area[0].get_attribute('innerText')
print('supplyAreaChar >> ', supplyAreaChar)
supplyAreaInt = supplyAreaChar.split('/')
supplyAreaInt = supplyAreaInt[1].replace('평', '')

supplyAreaInt = supplyAreaInt.strip()

supplyAreaInt = int(supplyAreaInt)
print('supplyAreaInt >> ', supplyAreaInt)

supplyAreaChar >>  26 / 31평
supplyAreaInt >>  31


### 매매에서는 shortRent가 None

In [14]:
try:
    if post_type=='아파트':
        unrefined_shortRent = driver.find_elements_by_xpath(
            '/html/body/div[1]/div/div[5]/div[3]/div/table/tbody/tr/td[5]/p')
    else:
        unrefined_shortRent = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[2]/div/table/tbody/tr/td[5]/p')
        if not unrefined_shortRent:
            unrefined_shortRent = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[3]/div/table/tbody/tr/td[4]/p')
    shortRent = unrefined_shortRent[0].get_attribute('innerText')
    shortRent = shortRent

    if shortRent == '불가능':
        shortRent = False
    else:
        shortRent = True
        
except IndexError:
    # 매매에서는 단기임대가 없음. 
    shortRent=None
print('shortRent >>', shortRent)

shortRent >> False


## management가 안보이면 None

In [15]:
try:
    if post_type=="아파트":
        management = driver.find_element_by_xpath('/html/body/div[1]/div/div[5]/div[3]/div/table/tbody/tr/td[3]')
    else:
        if salesType == "매매":
            management = driver.find_element_by_xpath('/html/body/div[1]/div/div[5]/div[2]/div/table/tbody/tr/td[2]')
        else:
            management = driver.find_element_by_xpath('/html/body/div[1]/div/div[5]/div[2]/div/table/tbody/tr/td[3]')
except NoSuchElementException:
    management = None
#     management = driver.find_element_by_xpath('/html/body/div[1]/div/div[5]/div[2]/div/table/tbody/tr/td[3]')
try:
    management = management.get_attribute('innerText')
    management = management.replace('\n', '')
    management = management.replace(' ', '')
    management = management.replace('(', ' ')
    management = management.replace(')', ' ')
    management = management.replace(',', ' ')
    management = management.strip(' ')
    management = management.split(' ')
except AttributeError:
    pass

In [16]:
try:
    managementPay = management.pop(0)
    if managementPay.find('만원'):
        managementPay = managementPay.replace('만원', ' ')
        if managementPay == '없음':
            managementPay = 0
        managementPay = int(managementPay)
    else:
        managementPay = 0
    totalFee = managementPay
    print('totalFee >> ', totalFee)
    print('management >>', management)
except IndexError:
    pass
except AttributeError:
    pass
except NameError:
    pass

totalFee >>  18
management >> ['전기세', '수도세', '청소비', '유선TV']


In [17]:
try:
    if post_type =="아파트":
        if salesType =="매매":
            unrefined_parking = None
        else:
            unrefined_parking = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[3]/div/table/tbody/tr/td[4]/p')
    else:
        if salesType == "매매":
        # 일반 주택 매매
            unrefined_parking = driver.find_elements_by_xpath("/html/body/div[1]/div/div[5]/div[2]/div/table/tbody/tr/td[3]")
        else:    
            unrefined_parking = driver.find_elements_by_xpath("/html/body/div[1]/div/div[5]/div[2]/div/table/tbody/tr/td[4]/p")
    parkingDetail = unrefined_parking[0].get_attribute('innerText')
except IndexError:
    unrefined_parking = driver.find_elements_by_xpath(
        '/html/body/div[1]/div/div[5]/div[3]/div/table/tbody/tr/td[4]/p'
    )
    parkingDetail = unrefined_parking[0].get_attribute('innerText')
except TypeError:
    parkingDetail = True

In [18]:
parkingDetail

'가능(무료)'

In [19]:
if parkingDetail == '불가':
    parkingTF = False
else:
    parkingTF = True
print('parkingTF >>', parkingTF)

parkingTF >> True


In [20]:
try:
    if not salesType == "매매":
        if post_type =="아파트":
            unrefined_living_expenses = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[3]/div/div/div/label')
            unrefined_living_expenses_detail = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[3]/div/div/div/span')
        else:
            try:
                unrefined_living_expenses = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[2]/div/div/div/label')
                unrefined_living_expenses_detail = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[2]/div/div/div/span')
            except NoSuchElementException:
                unrefined_living_expenses = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[3]/div/div/div/label')
                unrefined_living_expenses_detail = driver.find_elements_by_xpath('/html/body/div[1]/div/div[5]/div[3]/div/table/tbody/tr/td[3]/p[2]/span')                
    else:
        living_expenses = None
        living_expenses_detail = None
except NoSuchElementException:
    pass

In [21]:
try:
    unrefined_living_expenses = unrefined_living_expenses[0].get_attribute('innerText')
    print(unrefined_living_expenses)
    living_expenses = unrefined_living_expenses.replace(' ', '')
    living_expenses_detail = unrefined_living_expenses_detail[0].get_attribute('innerText')
    print(living_expenses_detail)
except IndexError:
    unrefined_living_expenses = driver.find_elements_by_xpath(
        '/html/body/div[1]/div/div[5]/div[3]/div/div/div/label'
    )
    unrefined_living_expenses_detail = driver.find_elements_by_xpath(
        '/html/body/div[1]/div/div[5]/div[3]/div/div/div/span')
    unrefined_living_expenses = unrefined_living_expenses[0].get_attribute('innerText')
    living_expenses = unrefined_living_expenses.replace(' ', '')
    living_expenses_detail = unrefined_living_expenses_detail[0].get_attribute('innerText')
except TypeError:
    pass
except NameError:
    pass

18만 원 + α
(관리비)


In [22]:
print('living_expenses >>', living_expenses)
print('living_expenses_detail >> ', living_expenses_detail)

living_expenses >> 18만원+α
living_expenses_detail >>  (관리비)


In [23]:
if salesType == "매매":
    if post_type =="아파트":
        moveIn = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[7]/div')
    else:
        moveIn = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[7]/div')
else:
    if post_type =="아파트":
        moveIn = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[9]/div')
    else:
        moveIn = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[9]/div')
moveIn.get_attribute('innerText')

'2020.04.15'

In [24]:
moveIn.get_attribute('innerText')

'2020.04.15'

In [25]:
# option & sceurity

try:
    option_tag = driver.find_element_by_name('option')

    option_tag = option_tag.get_attribute('innerText')

    option_tag = option_tag.split('보안/안전시설')


    option = option_tag[0]
    option = option.split('\n\n')
    print(option)
    del option[0]
    del option[-1]
    
    print('result option', option)
    
    security = option_tag[1]
    security = security.split('\n\n')
    del security[-1]
    del security[0]
    print('result security', security)
except IndexError:
    print('안전 시설 없음.')
    security = None
except NoSuchElementException:
    print('옵션, 안전시설 없음')
    option = None
    security = None

['옵션', '책상', '옷장', '신발장', '가스레인지']
result option ['책상', '옷장', '신발장']
안전 시설 없음.


In [34]:
heatingType = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[3]/div')
heatingType = heatingType.get_attribute('innerText')
print(heatingType)

개별난방


In [26]:
if salesType=="매매":
    if post_type=="아파트":
        pet = True
    else:
        pet = True
else:
    if post_type=="아파트":
        pet = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[7]/div')
    else:
        pet = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[6]/div')
    pet = pet.get_attribute('innerText')
    if pet =="불가능":
        pet = False
    else:
        pet= True


pet >> False


In [27]:
if post_type=="아파트":
    elevator = True
else:
    elevator = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[5]/div')
    elevator = elevator.get_attribute('innerText')
    if elevator=="있음":
        elevator = True
    else:
        elevator=False
print('elevator >> ', elevator)

elevator >>  True


In [28]:
if post_type=="아파트":
    builtIn = True
else:
    builtIn = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[4]/div')
    builtIn = builtIn.get_attribute('innerText')
    if builtIn =="아님":
        builtIn = False
    else:
        builtIn = True
print('builtIn >>', builtIn)

builtIn >> True


In [29]:
#베란다
if post_type =="아파트":
    veranda = True
else:
    if salesType =='매매':
        veranda = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[6]/div')
        veranda = veranda.get_attribute('innerText')
        if veranda == "있음":
            veranda = True
        else:
            veranda = False
    else:
        veranda = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[7]/div')
        veranda = veranda.get_attribute('innerText')
        if veranda=='있음':
            veranda = True
        else:
            veranda = False
print('veranda >>', veranda)

veranda >> True


In [30]:
# depositLoan 전세 대출 자금
if post_type == '아파트':
    if salesType == '매매':
        depositLoan = True
    else:
        depositLoan = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[8]/div')
        depositLoan = depositLoan.get_attribute('innerText')
        if depositLoan == '가능':
            depositLoan = True
        else:
            depositLoan = False
else:
    if salesType == '매매':
            depositLoan = True
    else:
        depositLoan = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[8]/div')
        depositLoan = depositLoan.get_attribute('innerText')
        if depositLoan == '가능':
            depositLoan = True
        else:
            depositLoan = False
print('depositLoan >>', depositLoan)

depositLoan >> False


In [31]:
# totalCitizen 
if post_type == '아파트':
    totalCitizen = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[5]/div')
    totalCitizen = totalCitizen.get_attribute('innerText')
else:
    totalCitizen = None
print('totalCitizen >>', totalCitizen)

totalCitizen >> 636세대


In [32]:
if post_type=="아파트":
    totalPark = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[6]/div')
    totalPark = totalPark.get_attribute('innerText')
else:
    totalPark = None

totalPark >> 0.8대


In [33]:
if post_type == '아파트':
    complete = driver.find_element_by_xpath('/html/body/div[1]/div/div[2]/div/ul/li[4]/div')
    complete = complete.get_attribute('innerText')
else:
    complete = None
print('complete >> ', complete)

complete >>  1995.05 준공
